## Import Packages:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
from astropy.io import ascii
import corner
import os
from timeit import default_timer as timer
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.lines as mlines
from scipy.optimize import least_squares, curve_fit
from scipy.stats import f
from astropy.table import Table, vstack
import pandas as pd
import lmfit
from matplotlib import rcParams
from matplotlib.colors import LinearSegmentedColormap

## Define relations to calculate B, R, and Mdot:

SSA dominant absorption:

In [3]:
# B field calculation for SSA dominated absorption
def B_peak_SSA(p, F_p, D, nu_p, F_p_scale, D_scale, nu_p_scale):
    power = (-4 / ((2 * p) + 13))
    B = 0.58 * ((p - 2)**power) * ((F_p/F_p_scale)**(power/2)) * ((D/D_scale)**power) * (nu_p/nu_p_scale)
    return B

# shock radius for SSA dominated absorption
def R_peak_SSA(p, F_p, D, nu_p, F_p_scale, D_scale, nu_p_scale):
    R = 8.8*10**15 * ((p - 2)**(-1 / ((2 * p) + 13))) * ((F_p/F_p_scale)**((p + 6)/((2 * p) + 13))) * ((D/D_scale)**(((2 * p) + 12)/((2 * p) + 13))) * ((nu_p/nu_p_scale)**(-1))
    return R

# mass loss rate at a given epoch for SSA dominated absorption
def Mdot_peak_SSA(m_H, B, t, v_wind, B_scale, t_scale, v_wind_scale):
    Mdot = ((6*10**(-7))/(m_H**2)) * ((B/B_scale)**2) * ((t/t_scale)**2) * ((v_wind/v_wind_scale))
    return Mdot

FFA dominant absorption:

In [5]:
# mass loss rate at a given epoch for FFA dominated absorption, assuming fully, singly ionized wind
def Mdot_peak_FFA(tau, v_ej, t, T_e, v_wind, v_ej_scale, t_scale, T_e_scale, v_wind_scale):
    Mdot = (4.76 * 10**(-5)) * (tau**(0.5)) * ((v_ej/v_ej_scale)**1.5) * ((t/t_scale)**1.5) * ((T_e/T_e_scale)**0.675) * (v_wind/v_wind_scale)
    return Mdot

Mass loss rate considering multiple absorption components from Chandra et al. 2020:

In [28]:
def Mdot_peak_mult(nu_p, T_e, v_ej, t):
    vw1 = 2
    Mdot_scale = 10**-3
    Mdot = Mdot_scale * vw1 * (7.5*10**-2) * ((nu_p)**1.06) * ((T_e/(10**4))**0.67) * (v_ej/(10**4)) * ((t/1000)**1.5)
    return Mdot 

Synchrotron cooling dominant:

In [57]:
def nu_syn(epsilon_B, Mdot, v_wind, t, epsilon_B_scale, Mdot_scale, v_wind_scale):
    nu_syn = 240.0 * ((epsilon_B/epsilon_B_scale)**(-3.0/2.0)) * ((Mdot/Mdot_scale)**(-3.0/2.0)) * ((v_wind/v_wind_scale)**(3.0/2.0)) * (t/60.0)
    return nu_syn

def t_syn_ratio(epsilon_B, Mdot, v_wind, nu, t, epsilon_B_scale, Mdot_scale, v_wind_scale, nu_scale):
    t_syn_ratio = 2.0 * ((epsilon_B/epsilon_B_scale)**(-3.0/4.0)) * ((Mdot/Mdot_scale)**(-3.0/4.0)) * ((v_wind/v_wind_scale)**(3.0/4.0)) * ((nu/nu_scale)**(-0.5)) * ((t/10.0)**(0.5))
    return t_syn_ratio

Inverse Compton cooling dominant:

In [59]:
def nu_IC(epsilon_B, Mdot, v_wind, v_ej, L_bol, t, epsilon_B_scale, Mdot_scale, v_wind_scale, v_ej_scale, L_bol_scale):
    nu_IC = 8.0 * ((epsilon_B/epsilon_B_scale)**(0.5)) * ((Mdot/Mdot_scale)**(0.5)) * ((v_wind/v_wind_scale)**(-0.5)) * ((v_ej/v_ej_scale)**(4.0)) * (L_bol/L_bol_scale) * (t/60.0)
    return nu_IC

def t_IC_ratio(L_bol, epsilon_B, Mdot, v_wind, v_ej, nu, t, L_bol_scale, epsilon_B_scale, Mdot_scale, v_wind_scale, v_ej_scale, nu_scale):
    t_IC_ratio = 0.18 * ((L_bol/L_bol_scale)**(-1.0)) * ((epsilon_B/epsilon_B_scale)**(1.0/4.0)) * ((Mdot/Mdot_scale)**(1.0/4.0)) * ((v_wind/v_wind_scale)**(-1.0/4.0)) * ((v_ej/v_ej_scale)**(2.0)) * ((nu/nu_scale)**(-0.5)) * ((t/10.0)**(0.5))
    return t_IC_ratio

## Load in physical parameter values to use in calculations:

Load in model-independent parameters:

In [60]:
# physical values:
D = 10.1 # in Mpc, as adopted by Maeda et al. 2023 and Boestrom et al. 2020
D_err = 1.8 # ^^
v_wind = 20 # km/s, use 20 b/c it is used in Maeda+2023a
v_ej = 7000 # km/s, adopted from Boestrom et al. 2020
T_e = 10**5 # K, standard value to adopt
epsilon_B = 0.012 # from Maeda+2023a, Model A
m_H = 1 # fractional mass, normalize so that it's 1
#m_H = 1.67 * 10**(-24)# mass of Hydrogen in grams
# L_bol = ??

# scale factors:
F_p_scale = 1 # Jy
D_scale = 1 # Mpc
nu_p_scale = 10 # GHz, consistent with scale used for fitting
v_wind_scale = 10 # km/s, should this be the same or different from the actual value adopted?
v_ej_scale = 10**4 # km/s
T_e_scale = 10**5 # K
t_scale = 100 # days, adapt to be consistent for whatever formula
epsilon_B_scale = 0.1
B_scale = 1 # Gauss
# Mdot_scale = ??
L_bol_scale = 10.0**42.0 # erg/s

Load in the parameter info for SSA dominant absorption:

In [25]:
# both epochs:
p = 3.30

# ~1360 days:
nu_p_SSA_early = 11.91
F_p_SSA_early = 7.46

# ~1700 days:
nu_p_SSA_late = 11.91
F_p_SSA_late = 6.05

Load in the parameter info for FFA dominant absorption:

In [27]:
# define a function that just returns tau:
def tau_FFA(freq, K2):
    #scale the freqs by 10
    tau = K2 * (freq/10.0)**(-2.1)
    return tau

# both epochs:
alpha = 1.20

# ~1360 days:
K1_FFA_early = 13.71
K2_FFA_early = 0.65
tau_early_10GHz = tau_FFA(10, K2_FFA_early) # choose tau at 10 GHz, our scaling factor

# peak freq and flux density at 1360 days:
nu_p_FFA_early = 10.92
F_p_FFA_early = 7.22

# ~1700 days:
K1_FFA_late = 11.30
K2_FFA_late = 0.66
tau_late_10GHz = tau_FFA(10, K2_FFA_late) # choose tau at 10 GHz, our scaling factor

# peak freq and flux density at 1700 days:
nu_p_FFA_late = 10.92
F_p_FFA_late = 5.89

## Calculations:

Calculate B, R, Mdot for SSA:

In [62]:
# ~1360 days:
t_early = 1360
B_SSA_early = B_peak_SSA(p, F_p_SSA_early, D, nu_p_SSA_early, F_p_scale, D_scale, 5) # scale to 5 GHz, as in ref formula (Chandra 2018)
R_SSA_early = R_peak_SSA(p, F_p_SSA_early, D, nu_p_SSA_early, F_p_scale, D_scale, 5) # scale to 5 GHz, as in ref formula (Chandra 2018)
Mdot_SSA_early = Mdot_peak_SSA(m_H, B_SSA_early, t_early, v_wind, B_scale, t_scale, v_wind_scale)

physical_params_SSA_early = {
    'B':(B_SSA_early, 'Gauss'),
    'R':(R_SSA_early, 'cm'),
    'Mdot':(Mdot_SSA_early, 'solar masses per year')
}

# ~1700 days:
t_late = 1700
B_SSA_late = B_peak_SSA(p, F_p_SSA_late, D, nu_p_SSA_late, F_p_scale, D_scale, 5) # scale to 5 GHz, as in ref formula (Chandra 2018)
R_SSA_late = R_peak_SSA(p, F_p_SSA_late, D, nu_p_SSA_late, F_p_scale, D_scale, 5) # scale to 5 GHz, as in ref formula (Chandra 2018)
Mdot_SSA_late = Mdot_peak_SSA(m_H, B_SSA_late, t_late, v_wind, B_scale, t_scale, v_wind_scale)

physical_params_SSA_late = {
    'B':(B_SSA_late, 'Gauss'),
    'R':(R_SSA_late, 'cm'),
    'Mdot':(Mdot_SSA_late, 'solar masses per year')
}

print(physical_params_SSA_early)
print(physical_params_SSA_late)

{'B': (0.6654224933356119, 'Gauss'), 'R': (8.49011619817433e+16, 'cm'), 'Mdot': (9.827748122886751e-05, 'solar masses per year')}
{'B': (0.6798005366260846, 'Gauss'), 'R': (7.686752790844421e+16, 'cm'), 'Mdot': (0.00016026625729627866, 'solar masses per year')}


Calculate ejecta velocities based on R and t at each epoch:

In [36]:
convert_cm_to_km = (1/100) * (1/1000)
convert_days_to_s = (24) * (60) * (60)

R_early = physical_params_SSA_early['R'][0]
v_ej_early = (R_early / t_early) * convert_cm_to_km / convert_days_to_s

R_late = physical_params_SSA_late['R'][0]
v_ej_late = (R_late / t_late) * convert_cm_to_km / convert_days_to_s
v_ej_late

5233.355658254644

Calculate Mdot for FFA:

In [64]:
# ~1360 days:
# calculate at peak:
tau_peak = 1
t_early = 1360
Mdot_FFA_early = Mdot_peak_FFA(tau_peak, v_ej_early, t_early, T_e, v_wind, v_ej_scale, t_scale, T_e_scale, v_wind_scale)

# ~1700 days:
# calculate at peak:
t_late = 1700
Mdot_FFA_late = Mdot_peak_FFA(tau_peak, v_ej_late, t_late, T_e, v_wind, v_ej_scale, t_scale, T_e_scale, v_wind_scale)
Mdot_FFA_late

0.0025262754749625907

Calculate Mdot from Chandra et al. 2020, include multiple absorption components:

In [46]:
# ~1360 days 
Mdot_early = Mdot_peak_mult(nu_p_FFA_early, T_e, v_ej_early, t_early)

# 1700 days
Mdot_late = Mdot_peak_mult(nu_p_FFA_late, T_e, v_ej_early, t_late)

Calculate synchrtron cooling values, both epochs:

In [56]:
# ~1360 days:
# break frequency:
nu_syn_early = nu_syn(epsilon_B, Mdot_early, v_wind, t_early, epsilon_B_scale, 10.0**-6.0, v_wind_scale) # Mass loss scaling of 10**-6, below as well
# ratio of cooling timescale to adiabatic timescale
t_ratio_syn_early = t_syn_ratio(epsilon_B, Mdot_early, v_wind, nu_syn_early, t_early, epsilon_B_scale, 10.0**-6.0, v_wind_scale, 10) # freq scale of 10 GHz

# ~1700 days:
# break frequency:
nu_syn_late = nu_syn(epsilon_B, Mdot_late, v_wind, t_late, epsilon_B_scale, 10.0**-6.0, v_wind_scale) # Mass loss scaling of 10**-6, below as well
# ratio of cooling timescale to adiabatic timescale
t_ratio_syn_late = t_syn_ratio(epsilon_B, Mdot_late, v_wind, nu_syn_late, t_late, epsilon_B_scale, 10.0**-6.0, v_wind_scale, 10) # freq scale of 10 GHz

Calculate IC cooling values, both epochs:

In [ ]:
def nu_IC(epsilon_B, Mdot, v_wind, v_ej, L_bol, t, epsilon_B_scale, Mdot_scale, v_wind_scale, v_ej_scale, L_bol_scale):
    nu_IC = 8.0 * ((epsilon_B/epsilon_B_scale)**(0.5)) * ((Mdot/Mdot_scale)**(0.5)) * ((v_wind/v_wind_scale)**(-0.5)) * ((v_ej/v_ej_scale)**(4.0)) * (L_bol/L_bol_scale) * (t/60.0)
    return nu_IC

def t_IC_ratio(L_bol, epsilon_B, Mdot, v_wind, v_ej, nu, t, L_bol_scale, epsilon_B_scale, Mdot_scale, v_wind_scale, v_ej_scale, nu_scale):
    t_IC_ratio = 0.18 * ((L_bol/L_bol_scale)**(-1.0)) * ((epsilon_B/epsilon_B_scale)**(1.0/4.0)) * ((Mdot/Mdot_scale)**(1.0/4.0)) * ((v_wind/v_wind_scale)**(-1.0/4.0)) * ((v_ej/v_ej_scale)**(2.0)) * ((nu/nu_scale)**(-0.5)) * ((t/10.0)**(0.5))
    return t_IC_ratio

In [61]:
# ~1360 days:
# break frequency:
nu_IC_early = nu_IC(epsilon_B, Mdot_early, v_wind, v_ej_early, L_bol, t_early, epsilon_B_scale, 10.0**-6.0, v_wind_scale, v_ej_scale, L_bol_scale) # Mass loss scaling of 10**-6, below as well
# ratio of cooling timescale to adiabatic timescale
t_ratio_IC_early = t_IC_ratio(L_bol, epsilon_B, Mdot_early, v_wind, v_ej_early, nu_IC_early, t_early, L_bol_scale, epsilon_B_scale, 10.0**-6.0, v_wind_scale, v_ej_scale, 10) # freq scale of 10 GHz

# ~1700 days:
# break frequency:
nu_IC_late = nu_IC(epsilon_B, Mdot_late, v_wind, v_ej_late, L_bol, t_late, epsilon_B_scale, 10.0**-6.0, v_wind_scale, v_ej_scale, L_bol_scale) # Mass loss scaling of 10**-6, below as well
# ratio of cooling timescale to adiabatic timescale
t_ratio_IC_late = t_IC_ratio(L_bol, epsilon_B, Mdot_late, v_wind, v_ej_late, nu_IC_late, t_early, L_bol_scale, epsilon_B_scale, 10.0**-6.0, v_wind_scale, v_ej_scale, 10) # freq scale of 10 GHz

NameError: name 'L_bol' is not defined